In [1]:
import psycopg2

In [26]:
connection = psycopg2.connect(
    host='localhost',
    port=54320,
    dbname='ht_db',
    user='postgres'
)

In [3]:
def create_table(name: str, schema: str, connection: psycopg2.extensions.connection):
    c = connection.cursor()
    ddl = f"""CREATE TABLE IF NOT EXISTS {name} ({schema})"""
    c.execute(ddl)
    connection.commit()
    c.close()

In [4]:
def insert_values(table_name: str, source: str, connection: psycopg2.extensions.connection):
    c = connection.cursor()
    with open(source, 'r') as f:
        next(f) # to skip header
        c.copy_expert(f"COPY {table_name} FROM STDIN CSV NULL AS ''", f)
    connection.commit()
    c.close()

In [5]:
def get_values(query: str, connection: psycopg2.extensions.connection):
    c = connection.cursor()
    c.execute(query)
    result = c.fetchone()
    print(result)
    c.close()

In [6]:
def drop_table(name: str, connection: psycopg2.extensions.connection):
    c = connection.cursor()
    ddl = f"""DROP TABLE {name}"""
    c.execute(ddl)
    connection.commit()
    c.close()

In [7]:
schema = '''complaint_id integer primary key,
complaint_text text not null,
was_user_disputed varchar(3),
date timestamp without time zone not null,
product_id integer,
issue_id integer
    '''
create_table('complaints_users', schema, connection=connection)

In [8]:
table_name = 'complaints_users'
source = 'data/complaints_users.csv'
insert_values(table_name, source, connection=connection)

In [28]:
query = "SELECT * FROM issues LIMIT 100"
get_values(query, connection)

(253, "Problem with a credit reporting company's investigation into an existing problem", 'Their investigation did not fix an error on your report')


In [29]:
drop_table('complaints_users', connection)
drop_table('complaints_companies', connection)
drop_table('products', connection)
drop_table('issues', connection)